In [ ]:
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from dotenv import load_dotenv
import os
import pandas as pd


In [ ]:
load_dotenv(r"C:\Users\kwasi\OneDrive\Documents\Personal Projects\schwab_trader\venv\.env")  # or just ".env" depending on your setup

API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET")
#print(API_KEY, API_SECRET)


In [ ]:
# Initialize Alpaca historical data client (for stocks)
client = StockHistoricalDataClient(API_KEY, API_SECRET)

In [ ]:
from datetime import datetime, timedelta

def get_price_data(symbol: str, timeframe: TimeFrame = TimeFrame.Day, limit: int = 30) -> pd.DataFrame:
    """
    Fetch historical OHLCV data for a symbol using alpaca-py.

    :param symbol: Ticker symbol, e.g. "AAPL"
    :param timeframe: TimeFrame object, e.g. TimeFrame.Minute, TimeFrame.Day
    :param limit: Number of bars to fetch
    :return: pandas DataFrame with OHLCV data
    """
    end = datetime.now()
    start = end - timedelta(days=limit * 2)  # buffer for weekends/holidays

    request = StockBarsRequest(
        symbol_or_symbols=[symbol],
        timeframe=timeframe,
        start=start,
        end=end
    )

    bars = client.get_stock_bars(request).df
    symbol_df = bars[bars.index.get_level_values("symbol") == symbol].copy()
    symbol_df.index = symbol_df.index.droplevel("symbol")
    symbol_df.index = symbol_df.index.tz_convert("America/New_York")

    return symbol_df

In [ ]:
df = get_price_data("AAPL", timeframe=TimeFrame.Day, limit=30)
df[["open", "high", "low", "close", "volume"]].head() # interacting with the API and get price data

In [ ]:
from alpaca.data.requests import StockLatestQuoteRequest, StockLatestTradeRequest

# Latest quote
quote = client.get_stock_latest_quote(StockLatestQuoteRequest(symbol_or_symbols=["AAPL"]))
print(quote)

# Latest trade
trade = client.get_stock_latest_trade(StockLatestTradeRequest(symbol_or_symbols=["AAPL"]))
print(trade)

In [ ]:
from alpaca.trading.client import TradingClient

trading_client = TradingClient(API_KEY, API_SECRET, paper=True)

account = trading_client.get_account()
print(account.buying_power)
print(account.status)

In [ ]:
positions = trading_client.get_all_positions()
for p in positions:
    print(f"{p.symbol}: {p.qty} shares @ avg price {p.avg_entry_price}")

In [ ]:
orders = trading_client.get_orders()  # open, closed, all
for o in orders:
    print(f"{o.symbol} | {o.side} | {o.qty} | {o.status}")

In [ ]:
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
# placing orders 
order = MarketOrderRequest(
    symbol="AAPL",
    qty=1,
    side=OrderSide.BUY,
    time_in_force=TimeInForce.DAY
)

response = trading_client.submit_order(order)
print(response)


In [ ]:
from alpaca.trading.client import TradingClient

calendar = trading_client.get_calendar()
for day in calendar[:5]:
    print(f"{day.date} | Open: {day.open} | Close: {day.close}")

In [ ]:
import asyncio
import nest_asyncio
from alpaca.data.live import StockDataStream
stream = StockDataStream(API_KEY, API_SECRET)

# ✅ Handler must be an async function
async def handle_quote(data):
    print("Quote:", data)

# ✅ Subscribe correctly — NOT using @decorator
stream.subscribe_quotes(handle_quote, "SPY")

# Run the stream
async def main():
    await stream.run()

nest_asyncio.apply()

asyncio.run(main())

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent  # Adjust if needed
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

import os
import asyncio
import logging
import pandas as pd
from datetime import datetime
from collections import deque
from dotenv import load_dotenv

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.live import StockDataStream

from strategies.strategy_registry.momentum_strategy import MomentumStrategy
from core.position_sizer import DynamicPositionSizer
from indicators.atr import ATRIndicator

#environment variables
API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET")

# Logging setup
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("LiveRunner")

# Trading client
trading_client = TradingClient(API_KEY, API_SECRET, paper=True)

# Stream client
stream = StockDataStream(API_KEY, API_SECRET)

# Configuration
symbol = "AAPL"
live_buffer = deque(maxlen=100)
strategy = MomentumStrategy()
sizer = DynamicPositionSizer(risk_percentage=0.07)

portfolio_history = []
total_fees = 0
peak_portfolio_value = 0


def submit_order(symbol, qty, side):
    order = MarketOrderRequest(
        symbol=symbol,
        qty=int(qty),
        side=side,
        time_in_force=TimeInForce.DAY
    )
    try:
        response = trading_client.submit_order(order)
        return response
    except Exception as e:
        logger.error(f"Order submission failed: {e}")
        return None


def evaluate_trade_opportunity(df, cash, position):
    global total_fees, peak_portfolio_value

    df = ATRIndicator(df)
    df = strategy.generate_signal(df)

    latest = df.iloc[-1]
    signal = latest['Signal']
    price = latest['Close']
    atr_value = latest['ATR']

    if pd.isna(atr_value) or atr_value <= 0:
        return None

    atr_25 = df['ATR'].quantile(0.25)
    atr_75 = df['ATR'].quantile(0.75)

    market_conditions = (
        "low_volatility" if atr_value < atr_25 else
        "high_volatility" if atr_value > atr_75 else
        "normal"
    )

    stop_loss_price = price - (atr_value * 2)
    quantity = sizer.calculate_position_size(price, stop_loss_price, cash, market_conditions)
    trade_fee = 0.001 * price * quantity
    max_affordable_qty = cash // (price + trade_fee)
    quantity = min(quantity, max_affordable_qty)

    return {
        "signal": signal,
        "price": price,
        "quantity": quantity,
        "stop_loss": stop_loss_price,
        "fee": trade_fee,
        "market_conditions": market_conditions
    }


async def on_bar(bar):
    # Update buffer
    live_buffer.append({
        "Date": bar.timestamp,
        "Open": bar.open,
        "High": bar.high,
        "Low": bar.low,
        "Close": bar.close,
        "Volume": bar.volume
    })

    if len(live_buffer) < 10:
        return

    try:
        account = trading_client.get_account()
        cash = float(account.cash)
    except Exception as e:
        logger.error(f"Failed to fetch account info: {e}")
        return

    try:
        position_data = trading_client.get_open_position(symbol)
        position = int(float(position_data.qty))
    except:
        position = 0

    df = pd.DataFrame(live_buffer)
    trade_plan = evaluate_trade_opportunity(df, cash, position)
    if not trade_plan:
        return

    signal = trade_plan["signal"]
    price = trade_plan["price"]
    quantity = trade_plan["quantity"]
    stop_loss_price = trade_plan["stop_loss"]

    if signal == 1 and quantity > 0 and position == 0:
        submit_order(symbol, quantity, OrderSide.BUY)
        logger.info(f"[LIVE] BUY {quantity} {symbol} @ {price:.2f}")

    elif signal == -1 and position > 0:
        submit_order(symbol, position, OrderSide.SELL)
        logger.info(f"[LIVE] SELL {position} {symbol} @ {price:.2f}")

    elif position > 0 and price <= stop_loss_price:
        submit_order(symbol, position, OrderSide.SELL)
        logger.warning(f"[LIVE] STOP-LOSS SELL {position} {symbol} @ {price:.2f}")

    portfolio_value = cash + position * price
    global peak_portfolio_value
    peak_portfolio_value = max(peak_portfolio_value, portfolio_value)
    drawdown = (portfolio_value - peak_portfolio_value) / peak_portfolio_value

    portfolio_history.append({
        "Date": datetime.now(),
        "Portfolio_Value": portfolio_value,
        "Cash": cash,
        "Position": position,
        "Price": price,
        "Drawdown": drawdown
    })


# Hook up the stream
stream.subscribe_bars(on_bar, symbol)


async def main():
    await stream.run()


if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())


In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent  # Adjust if needed
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

import os
import asyncio
import logging
import pandas as pd
from datetime import datetime
from collections import deque
from dotenv import load_dotenv
import requests

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.live import StockDataStream

# Your strategy and sizer
from strategies.strategy_registry.momentum_strategy import MomentumStrategy
from core.position_sizer import DynamicPositionSizer
from indicators.atr import ATRIndicator

# Load environment
load_dotenv(r"C:\Users\kwasi\OneDrive\Documents\Personal Projects\schwab_trader\venv\.env") 

API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET")
DISCORD_WEBHOOK = os.getenv("DISCORD_WEBHOOK")

# Logging setup
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("LiveRunner")

# Trading client
trading_client = TradingClient(API_KEY, API_SECRET, paper=True)

# Stream client
stream = StockDataStream(API_KEY, API_SECRET)

# Configuration
symbol = "AAPL"
live_buffer = deque(maxlen=100)
strategy = MomentumStrategy()
sizer = DynamicPositionSizer(risk_percentage=0.07)

portfolio_history = []
total_fees = 0
peak_portfolio_value = 0

async def send_heartbeat(interval_seconds=3600):
    await asyncio.sleep(10)  # slight delay after boot
    send_discord_message(":rocket: LLM Trading Bot is now running.")

    while True:
        await asyncio.sleep(interval_seconds)
        send_discord_message(":heartbeat: LLM Trading Bot is still running...")

def submit_order(symbol, qty, side):
    order = MarketOrderRequest(
        symbol=symbol,
        qty=int(qty),
        side=side,
        time_in_force=TimeInForce.DAY
    )
    try:
        response = trading_client.submit_order(order)
        return response
    except Exception as e:
        logger.error(f"Order submission failed: {e}")
        return None

def send_discord_message(content):
    if not DISCORD_WEBHOOK:
        logger.warning("No Discord webhook configured.")
        return
    try:
        requests.post(DISCORD_WEBHOOK, json={"content": content})
    except Exception as e:
        logger.error(f"Failed to send Discord message: {e}")

def query_local_llm(prompt: str):
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={"model": "deepseek", "prompt": prompt, "stream": False}
        )
        result = response.json()
        return result.get("response", "").strip()
    except Exception as e:
        logger.error(f"Failed to query LLM: {e}")
        return ""

async def on_bar(bar):
    global total_fees, peak_portfolio_value

    # Update buffer
    live_buffer.append({
        "Date": bar.timestamp,
        "Open": bar.open,
        "High": bar.high,
        "Low": bar.low,
        "Close": bar.close,
        "Volume": bar.volume
    })

    if len(live_buffer) < 10:
        return

    # Fetch latest account state
    try:
        account = trading_client.get_account()
        cash = float(account.cash)
    except Exception as e:
        logger.error(f"Failed to fetch account info: {e}")
        return

    try:
        position_data = trading_client.get_open_position(symbol)
        position = int(float(position_data.qty))
    except:
        position = 0

    df = pd.DataFrame(live_buffer)
    df = ATRIndicator(df)
    df = strategy.generate_signal(df)

    latest = df.iloc[-1]
    signal = latest['Signal']
    price = latest['Close']
    atr_value = latest['ATR']

    if pd.isna(atr_value) or atr_value <= 0:
        return

    atr_25 = df['ATR'].quantile(0.25)
    atr_75 = df['ATR'].quantile(0.75)

    market_conditions = (
        "low_volatility" if atr_value < atr_25 else
        "high_volatility" if atr_value > atr_75 else
        "normal"
    )

    stop_loss_price = price - (atr_value * 2)
    quantity = sizer.calculate_position_size(price, stop_loss_price, cash, market_conditions)
    trade_fee = 0.001 * price * quantity
    max_affordable_qty = cash // (price + trade_fee)
    quantity = min(quantity, max_affordable_qty)

    llm_message = None
    if signal == 1 and quantity > 0 and position == 0:
        submit_order(symbol, quantity, OrderSide.BUY)
        message = f"[LIVE] BUY {quantity} {symbol} @ {price:.2f}"
        logger.info(message)
        llm_message = message

    elif signal == -1 and position > 0:
        submit_order(symbol, position, OrderSide.SELL)
        message = f"[LIVE] SELL {position} {symbol} @ {price:.2f}"
        logger.info(message)
        llm_message = message

    elif position > 0 and price <= stop_loss_price:
        submit_order(symbol, position, OrderSide.SELL)
        message = f"[LIVE] STOP-LOSS SELL {position} {symbol} @ {price:.2f}"
        logger.warning(message)
        llm_message = message

    portfolio_value = cash + position * price
    peak_portfolio_value = max(peak_portfolio_value, portfolio_value)
    drawdown = (portfolio_value - peak_portfolio_value) / peak_portfolio_value

    portfolio_history.append({
        "Date": datetime.now(),
        "Portfolio_Value": portfolio_value,
        "Cash": cash,
        "Position": position,
        "Price": price,
        "Drawdown": drawdown
    })

    # Send LLM summary to Discord
    if llm_message:
        prompt = f"Analyze the following trade activity and provide a short summary with risk commentary:\n{llm_message}"
        llm_response = query_local_llm(prompt)
        if llm_response:
            send_discord_message(f"\U0001F916 LLM Insight:\n{llm_response}")

# Hook up the stream
stream.subscribe_bars(on_bar, symbol)

# Stream retry logic
stream_started = False
retry_cooldown_seconds = 300  # 5 minutes

async def run_stream_with_retry():
    global stream_started
    while not stream_started:
        try:
            logger.info("Attempting to start Alpaca stream...")
            await stream.run()
            stream_started = True
        except Exception as e:
            logger.error(f"Stream failed to start: {e}")
            send_discord_message(f"⚠️ Stream failed to start. Retrying in 5 minutes.\nError: {e}")
            await asyncio.sleep(retry_cooldown_seconds)

async def main():
    heartbeat_task = asyncio.create_task(send_heartbeat())
    await asyncio.gather(heartbeat_task, run_stream_with_retry())

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())



ERROR:LiveRunner:Stream failed to start: 


In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

import os
import asyncio
import logging
import pandas as pd
from datetime import datetime, date
from collections import deque
from dotenv import load_dotenv
import requests
import json

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.live import StockDataStream

from strategies.strategy_registry.momentum_strategy import MomentumStrategy
from core.position_sizer import DynamicPositionSizer
from indicators.atr import ATRIndicator

load_dotenv(r"C:\Users\kwasi\OneDrive\Documents\Personal Projects\schwab_trader\venv\.env")

API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET")
DISCORD_WEBHOOK = os.getenv("DISCORD_WEBHOOK")

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("LiveRunner")

trading_client = TradingClient(API_KEY, API_SECRET, paper=True)
stream = StockDataStream(API_KEY, API_SECRET)

symbol = "AAPL"
daily_buffer = deque(maxlen=100)
current_day_bar = None
strategy = MomentumStrategy()
sizer = DynamicPositionSizer(risk_percentage=0.07)

portfolio_history = []
total_fees = 0
peak_portfolio_value = 0
last_trade_date = None

# Load historical daily bars
processed_path = Path(rf"C:\Users\kwasi\OneDrive\Documents\Personal Projects\schwab_trader\data\data_storage\proc_data\proc_{symbol}_file.json")
if processed_path.exists():
    with open(processed_path, "r") as f:
        processed_data = json.load(f)
        for item in processed_data[-100:]:
            daily_buffer.append({
                "Date": pd.to_datetime(item["Date"], unit='ms'),
                "Open": item["Open"],
                "High": item["High"],
                "Low": item["Low"],
                "Close": item["Close"],
                "Volume": item["Volume"],
                "ATR": item.get("ATR"),
                "RSI": item.get("RSI"),
                "Momentum": item.get("Momentum"),
                "VWAP": item.get("VWAP"),
                "OBV": item.get("OBV"),
                "MACD": item.get("MACD"),
                "MACD_Signal": item.get("MACD_Signal"),
                "Price_Change": item.get("Price_Change"),
                "Daily_Return": item.get("Daily_Return"),
                "Lag_Close_1": item.get("Lag_Close_1"),
                # Add more if your strategy uses them
            })
    logger.info(f"Loaded {len(daily_buffer)} preprocessed bars for {symbol}.")
else:
    logger.warning(f"Processed data not found for {symbol}.")

async def send_heartbeat(interval_seconds=3600):
    await asyncio.sleep(10)
    send_discord_message(":rocket: LLM Trading Bot is now running.")
    while True:
        await asyncio.sleep(interval_seconds)
        send_discord_message(":heartbeat: LLM Trading Bot is still running...")

def submit_order(symbol, qty, side):
    order = MarketOrderRequest(
        symbol=symbol,
        qty=int(qty),
        side=side,
        time_in_force=TimeInForce.DAY
    )
    try:
        return trading_client.submit_order(order)
    except Exception as e:
        logger.error(f"Order submission failed: {e}")
        return None

def send_discord_message(content):
    if not DISCORD_WEBHOOK:
        return
    try:
        requests.post(DISCORD_WEBHOOK, json={"content": content})
    except Exception as e:
        logger.error(f"Failed to send Discord message: {e}")

def query_local_llm(prompt: str):
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={"model": "deepseek", "prompt": prompt, "stream": False}
        )
        return response.json().get("response", "").strip()
    except Exception as e:
        logger.error(f"Failed to query LLM: {e}")
        return ""

async def on_bar(bar):
    global current_day_bar, peak_portfolio_value, last_trade_date

    timestamp = pd.to_datetime(bar.timestamp)
    bar_date = timestamp.date()

    # Finalize yesterday's bar if needed
    if current_day_bar and current_day_bar['Date'].date() != bar_date:
        daily_buffer.append(current_day_bar)
        logger.info(f"Finalized bar for {current_day_bar['Date'].date()}")
        current_day_bar = None

    # Create or update current day's bar
    if current_day_bar is None:
        current_day_bar = {
            "Date": timestamp,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume
        }
    else:
        current_day_bar["High"] = max(current_day_bar["High"], bar.high)
        current_day_bar["Low"] = min(current_day_bar["Low"], bar.low)
        current_day_bar["Close"] = bar.close
        current_day_bar["Volume"] += bar.volume

    logger.info(f"Minute update: {timestamp} | O: {bar.open:.2f} H: {bar.high:.2f} L: {bar.low:.2f} C: {bar.close:.2f} V: {bar.volume}")

    # Combine historical bars with the evolving current day
    df = pd.DataFrame(list(daily_buffer) + [current_day_bar])
    if len(df) < 20:
        return

    # Recompute indicators for most recent bars only
    df_tail = df.tail(20).copy()
    df_tail = ATRIndicator(df_tail).compute()
    df.update(df_tail)

    df = strategy.generate_signal(df)
    latest = df.iloc[-1]

    signal = latest['Signal']
    price = latest['Close']
    atr_value = latest['ATR']

    if pd.isna(atr_value) or atr_value <= 0:
        return

    atr_25 = df['ATR'].quantile(0.25)
    atr_75 = df['ATR'].quantile(0.75)

    market_conditions = (
        "low_volatility" if atr_value < atr_25 else
        "high_volatility" if atr_value > atr_75 else
        "normal"
    )

    try:
        account = trading_client.get_account()
        cash = float(account.cash)
    except Exception as e:
        logger.error(f"Failed to fetch account info: {e}")
        return

    try:
        position_data = trading_client.get_open_position(symbol)
        position = int(float(position_data.qty))
    except:
        position = 0

    stop_loss_price = price - (atr_value * 2)
    quantity = sizer.calculate_position_size(price, stop_loss_price, cash, market_conditions)
    trade_fee = 0.001 * price * quantity
    max_affordable_qty = cash // (price + trade_fee)
    quantity = min(quantity, max_affordable_qty)

    print("\n=== LIVE BAR UPDATE ===")
    print(df.tail(5).to_string(index=False))
    print(f"Signal: {signal} | ATR: {atr_value:.2f} | Market: {market_conditions} | Stop-loss: {stop_loss_price:.2f}")

    today_str = date.today().isoformat()
    if last_trade_date == today_str:
        return

    llm_message = None
    if signal == 1 and quantity > 0 and position == 0:
        submit_order(symbol, quantity, OrderSide.BUY)
        llm_message = f"[LIVE] BUY {quantity} {symbol} @ {price:.2f}"
        last_trade_date = today_str
    elif signal == -1 and position > 0:
        submit_order(symbol, position, OrderSide.SELL)
        llm_message = f"[LIVE] SELL {position} {symbol} @ {price:.2f}"
        last_trade_date = today_str
    elif position > 0 and price <= stop_loss_price:
        submit_order(symbol, position, OrderSide.SELL)
        llm_message = f"[LIVE] STOP-LOSS SELL {position} {symbol} @ {price:.2f}"
        last_trade_date = today_str

    if llm_message:
        logger.info(llm_message)
        prompt = f"Analyze the following trade activity and provide a short summary with risk commentary:\n{llm_message}"
        llm_response = query_local_llm(prompt)
        if llm_response:
            send_discord_message(f"🤖 LLM Insight:\n{llm_response}")

    portfolio_value = cash + position * price
    peak_portfolio_value = max(peak_portfolio_value, portfolio_value)
    drawdown = (portfolio_value - peak_portfolio_value) / peak_portfolio_value
    portfolio_history.append({
        "Date": datetime.now(),
        "Portfolio_Value": portfolio_value,
        "Cash": cash,
        "Position": position,
        "Price": price,
        "Drawdown": drawdown
    })


stream.subscribe_bars(on_bar, symbol)

stream_started = False
retry_cooldown_seconds = 300

async def run_stream_with_retry():
    global stream_started
    while not stream_started:
        try:
            logger.info("Attempting to start Alpaca stream...")
            await stream.run()
            stream_started = True
        except Exception as e:
            logger.error(f"Stream failed to start: {e}")
            send_discord_message(f"⚠️ Stream failed to start. Retrying in 5 minutes.\nError: {e}")
            await asyncio.sleep(retry_cooldown_seconds)

async def main():
    heartbeat_task = asyncio.create_task(send_heartbeat())
    await asyncio.gather(heartbeat_task, run_stream_with_retry())

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())


INFO:LiveRunner:Loaded 100 preprocessed bars for AAPL.
INFO:LiveRunner:Attempting to start Alpaca stream...
INFO:alpaca.data.live.websocket:started data stream
INFO:alpaca.data.live.websocket:starting data websocket connection
INFO:alpaca.data.live.websocket:connecting to wss://stream.data.alpaca.markets/v2/iex
INFO:alpaca.data.live.websocket:connected to wss://stream.data.alpaca.markets/v2/iex
INFO:alpaca.data.live.websocket:subscribed to bars: ['AAPL']
INFO:LiveRunner:Minute update: 2025-07-18 19:31:00+00:00 | O: 210.95 H: 210.97 L: 210.88 C: 210.88 V: 1774.0
INFO:LiveRunner:[LIVE] SELL 1 AAPL @ 210.88



=== LIVE BAR UPDATE ===
                     Date    Open   High    Low  Close     Volume      ATR       RSI  Momentum      VWAP         OBV      MACD  MACD_Signal  Price_Change  Daily_Return  Lag_Close_1  Signal
      2025-07-14 05:00:00 209.925 210.91 207.54 208.62 38840111.0 4.189693 42.367338      7.54 49.842942 196528724.0  0.124503     0.368313          0.06      0.000388        77.38       1
      2025-07-15 05:00:00 209.220 211.89 208.92 209.11 42296339.0 4.191836 39.773867      3.94 49.844792 195552592.0 -0.058092     0.283032         -0.94     -0.007880        77.41       1
      2025-07-16 05:00:00 210.295 212.40 208.64 210.16 47490532.0 4.219693 41.296449      2.34 49.846630 196546715.0 -0.181343     0.190157         -0.03      0.003125        76.80       1
      2025-07-17 05:00:00 210.570 211.80 209.59 210.02 48068141.0 4.150407 46.679779     -2.42 49.848909 197748757.0 -0.204843     0.111157          0.58      0.011552        77.04      -1
2025-07-18 19:31:00+00:00 210.

INFO:LiveRunner:Minute update: 2025-07-18 19:32:00+00:00 | O: 210.88 H: 210.91 L: 210.87 C: 210.90 V: 3617.0



=== LIVE BAR UPDATE ===
                     Date    Open   High     Low   Close     Volume      ATR       RSI  Momentum      VWAP         OBV      MACD  MACD_Signal  Price_Change  Daily_Return  Lag_Close_1  Signal
      2025-07-14 05:00:00 209.925 210.91 207.540 208.620 38840111.0 4.189693 42.367338     7.540 49.842942 196528724.0  0.124503     0.368313          0.06      0.000388        77.38       1
      2025-07-15 05:00:00 209.220 211.89 208.920 209.110 42296339.0 4.191836 39.773867     3.940 49.844792 195552592.0 -0.058092     0.283032         -0.94     -0.007880        77.41       1
      2025-07-16 05:00:00 210.295 212.40 208.640 210.160 47490532.0 4.219693 41.296449     2.340 49.846630 196546715.0 -0.181343     0.190157         -0.03      0.003125        76.80       1
      2025-07-17 05:00:00 210.570 211.80 209.590 210.020 48068141.0 4.150407 46.679779    -2.420 49.848909 197748757.0 -0.204843     0.111157          0.58      0.011552        77.04      -1
2025-07-18 19:31:00+

INFO:LiveRunner:Minute update: 2025-07-18 19:33:00+00:00 | O: 210.90 H: 210.90 L: 210.85 C: 210.89 V: 2210.0


...


=== LIVE BAR UPDATE ===
                     Date    Open    High    Low  Close     Volume      ATR       RSI  Momentum      VWAP         OBV      MACD  MACD_Signal  Price_Change  Daily_Return  Lag_Close_1  Signal
      2025-07-14 05:00:00 209.925 210.910 207.54 208.62 38840111.0 4.189693 42.367338      7.54 49.842942 196528724.0  0.124503     0.368313          0.06      0.000388        77.38       1
      2025-07-15 05:00:00 209.220 211.890 208.92 209.11 42296339.0 4.191836 39.773867      3.94 49.844792 195552592.0 -0.058092     0.283032         -0.94     -0.007880        77.41       1
      2025-07-16 05:00:00 210.295 212.400 208.64 210.16 47490532.0 4.219693 41.296449      2.34 49.846630 196546715.0 -0.181343     0.190157         -0.03      0.003125        76.80       1
      2025-07-17 05:00:00 210.570 211.800 209.59 210.02 48068141.0 4.150407 46.679779     -2.42 49.848909 197748757.0 -0.204843     0.111157          0.58      0.011552        77.04      -1
2025-07-18 19:31:00+00:00

INFO:LiveRunner:Minute update: 2025-07-18 19:43:00+00:00 | O: 211.03 H: 211.03 L: 210.96 C: 211.03 V: 2379.0



=== LIVE BAR UPDATE ===
                     Date    Open    High    Low  Close     Volume      ATR       RSI  Momentum      VWAP         OBV      MACD  MACD_Signal  Price_Change  Daily_Return  Lag_Close_1  Signal
      2025-07-14 05:00:00 209.925 210.910 207.54 208.62 38840111.0 4.189693 42.367338      7.54 49.842942 196528724.0  0.124503     0.368313          0.06      0.000388        77.38       1
      2025-07-15 05:00:00 209.220 211.890 208.92 209.11 42296339.0 4.191836 39.773867      3.94 49.844792 195552592.0 -0.058092     0.283032         -0.94     -0.007880        77.41       1
      2025-07-16 05:00:00 210.295 212.400 208.64 210.16 47490532.0 4.219693 41.296449      2.34 49.846630 196546715.0 -0.181343     0.190157         -0.03      0.003125        76.80       1
      2025-07-17 05:00:00 210.570 211.800 209.59 210.02 48068141.0 4.150407 46.679779     -2.42 49.848909 197748757.0 -0.204843     0.111157          0.58      0.011552        77.04      -1
2025-07-18 19:31:00+00:00

INFO:LiveRunner:Minute update: 2025-07-18 19:44:00+00:00 | O: 210.92 H: 210.97 L: 210.86 C: 210.95 V: 3193.0



=== LIVE BAR UPDATE ===
                     Date    Open    High    Low  Close     Volume      ATR       RSI  Momentum      VWAP         OBV      MACD  MACD_Signal  Price_Change  Daily_Return  Lag_Close_1  Signal
      2025-07-14 05:00:00 209.925 210.910 207.54 208.62 38840111.0 4.189693 42.367338      7.54 49.842942 196528724.0  0.124503     0.368313          0.06      0.000388        77.38       1
      2025-07-15 05:00:00 209.220 211.890 208.92 209.11 42296339.0 4.191836 39.773867      3.94 49.844792 195552592.0 -0.058092     0.283032         -0.94     -0.007880        77.41       1
      2025-07-16 05:00:00 210.295 212.400 208.64 210.16 47490532.0 4.219693 41.296449      2.34 49.846630 196546715.0 -0.181343     0.190157         -0.03      0.003125        76.80       1
      2025-07-17 05:00:00 210.570 211.800 209.59 210.02 48068141.0 4.150407 46.679779     -2.42 49.848909 197748757.0 -0.204843     0.111157          0.58      0.011552        77.04      -1
2025-07-18 19:31:00+00:00